In [5]:
from __future__ import unicode_literals, print_function, division
import pandas as pd
import os
from io import open
import glob
import unicodedata
import string
import torch
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

print(torch.__version__)
print(pd.__version__)

torch.manual_seed(1)
# df = pd.read_fwf('sgns.sogou.bigram.txt')
# df = pd.read_csv("whitespace.csv", header=None, delim_whitespace=True)
from google.colab import drive
drive.mount('/content/drive')
cwd = '/content/drive/My Drive/nlp_project/'

1.5.0+cu101
1.0.4
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def tokenize(sentence, word_set):
  words = []
  start = 0
  while start < len(sentence):
    find = True
    for stop in range(len(sentence), start, -1):
      if sentence[start: stop].lower() in word_set:
        words.append(sentence[start: stop])
        start = stop
        break
      if stop == start + 1:
        find = False
        break
    if not find:
      start += 1
  return words

## get vocabulary set

In [ ]:
data = pd.read_excel(cwd + '2018 payroll.xlsx', header=None)

In [ ]:
word_set = set()
for i in range(5001):
  situation = str(data[31][i]).replace('，',' ').split() + str(data[31][i]).replace('，',' ').split()
  for w in situation:
    word_set.add(w)
print(word_set)
print(len(word_set))

{'此', '36', '0010', '省电', '地下室', '刷', '立', '非', '发货', '28', '查不到', '经济', '人数', '79', '卡', '梁', '地方', '深色', '米', '麟', '1861', '量', '产品', '......', '归属', '神隐', '定制', '开户', '打开', '银行卡', '？', '381', '看到', '咨询处', '不见', '网', '旧', '678', '评价', '663', '156', '武汉', '公里', '158', '103', 'Y', '一直', '短信', '文件', '耗电', '换手', '情况', '225', '新的', '个人', '咨询', '地位', '商城', '发起人', '供货价', '此人', '十月', '记得', '130', '高新', '代理商', '165', 'w', '半天', '26', '编辑', '区域', '考勤记录', '4', '5.0', '213', '年', '转', '存', '原来', 'a', '用', '清理', '室内', '换', '燕', '漂', '全部', '496', '但是', '跳转', '绑定', '录入', '50', '产业', '基站', '上午', '耗电量', '913', '自定义', '版', '调休', 'l', 'q', '重', '人事', '兑换', '最多', '清', '出差', '每天', '系统', '不对', '订货', '拒绝', '茶叶', '一百', '及', '缺失', '方法', '稍后', 'win', '输', '有', '飘', '大概', '通过', '份', '市场部', '填写', '天', '89', '成立', '没了', '参', '打叉', '再', '不在', '逐级', '待', '列表', '私下', '198', '司', '1318', '小数点', '北京', '偏差', '513', '插卡', '点选', '802', 'KEY', '安全', '断线', '新增', '会', '24', '排名', '安装包', '刚刚', '1370', '地板', '程序', '导', '有些',

In [ ]:
import pickle
with open(cwd + 'tokenized_word_set.pickle', 'wb') as f:
    pickle.dump(word_set, f)

## remove long words

In [ ]:
long_words = []
word_set_copy = word_set.copy()
for word in word_set_copy:
  if len(word) > 7:
    print(word)
    long_words.append(word)
    word_set.remove(word)

In [ ]:
print(len(long_words))
for w in long_words:
  words = tokenize(w, word_set)
  for word in words:
    word_set.add(word)
print(len(word_set))

0
2117


## remove numbers

In [ ]:
word_set_copy = word_set.copy()
for w in word_set_copy:
  if w.isdigit():
    word_set.remove(w)
print(len(word_set))

2171


## use bigram

In [ ]:
def load_embeddings(path):
    embedding_map = {}
    with open(path) as f:
        for line in f:
            try:
                pieces = line.rstrip().split()
                embedding_map[pieces[0]] = [float(weight) for weight in pieces[1:]]
            except:
                pass
    return embedding_map
d = load_embeddings(cwd + 'sgns.sogou.bigram.txt')

In [ ]:
print(len(d.keys()))
print(type(d.keys()))
word_set = set(d.keys())

364171
<class 'dict_keys'>


In [ ]:
problems = data['Unnamed: 31']
situation = data['Unnamed: 32']
for i in range(100):
  sent1 = data['Unnamed: 31'][i] 
  sent2 = data['Unnamed: 32'][i]
  print(i)
  print(sent1)
  word1 = tokenize(sent1, word_set)
  print(' '.join(word1))
  word2 = tokenize(sent2, word_set)
  print(sent2)
  print(' '.join(word2))
  print()
  problems[i] = ' '.join(word1)
  situation[i] = ' '.join(word2)

0
考勤 不
考勤 不
技术  紧急 处理
技术 紧急 处理

1
网络 科技 有限 公司    咨询 软件 ，七八十 人，外出 定位 轨迹  ，费用
网络 科技 有限 公司 咨询 软件 ， 七八十 人 ， 外出 定位 轨迹 ， 费用
转
转

2
华为 青春 
华为 青春
已调 手机 设置
已 调 手机 设置

3
魅族 定位 离线
魅族 定位 离线
已调 手机 设置
已 调 手机 设置

4
对接 红圈 定位 问题 
对接 红圈 定位 问题
转
转

5
配置
配置
已 处理
已 处理

6
配置
配置
已 配置
已 配置

7
配置
配置
已 配置
已 配置

8
配置
配置
已 配置
已 配置

9
定位 失联
定位 失 联
在线 调试 手机（未接到）
在线 调试 手机 （ 未接 到 ）

10
华为 手机 定位 失联
华为 手机 定位 失 联
已 提供 调试 手机 设置 步骤，建议 调试 之后 观察
已 提供 调试 手机 设置 步骤 ， 建议 调试 之后 观察

11
oppo  r9s  手机 设置 调试  找不到
o p p o r 9 s 手机 设置 调试 找 不到
在线 处理
在线 处理

12
可以 延期
可以 延期
建议 联系 开户
建议 联系 开户

13
配置
配置
已 配置
已 配置

14
调试 轨迹 不全
调试 轨迹 不 全
告知 手机 刚刚 调试，需要 调试 手机 定位 情况 是否 离线
告知 手机 刚刚 调试 ， 需要 调试 手机 定位 情况 是否 离线

15
新 配置、时间
新 配置 、 时间
新 员工 终端 配置、时间
新 员工 终端 配置 、 时间

16
反馈 客户  文件 错误
反馈 客户 文件 错误
客户 进入 在线 最终 核实  客户 WPS，告知 下载 office2013 以上 办公 软件
客户 进入 在线 最终 核实 客户 W P S ， 告知 下载 office 2013 以上 办公 软件

17
在线  调试
在线 调试
华为（1次） 进 在线后 没 说话 下线
华为 （ 1 次 ） 进 在线 后 没 说话 下线

18
灰点率  高
灰 点 率 高
告知 进入 在线 客服 核实 手机 设置，未 进入
告知 进入 在线 客服 核实 手机 设置 ， 未 进入

19
配置
配置
已 配置
已 配置

20


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [ ]:
print(len(data))

36951


## split new rows

In [ ]:
word_set = set([w.lower() for w in word_set])
print(len(word_set))

362553


In [ ]:
data = pd.read_excel(cwd + '2018_tagged_data.xlsx')
problems = data['Unnamed: 31']
situation = data['Unnamed: 32']
for i in range(len(data)):
  sent1 = str(data['Unnamed: 31'][i])
  sent2 = str(data['Unnamed: 32'][i])
  #print(i)
  #print(sent1)
  word1 = tokenize(sent1, word_set)
  #print(' '.join(word1))
  word2 = tokenize(sent2, word_set)
  #print(sent2)
  #print(' '.join(word2))
  #print()
  problems[i] = ' '.join(word1)
  situation[i] = ' '.join(word2)

data['Unnamed: 31'] = problems
data['Unnamed: 32'] = situation
data.to_excel(cwd + '2018 payroll.xlsx', header=False, index=False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
